# EDA

In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [4]:
AUDIO_DIR = "../input/train"
CSV_PATH = "../input/train.csv"
SR = 16000

In [5]:
df = pd.read_csv(CSV_PATH)
df["Target"] = df["Target"].astype(str)

In [6]:
def reconstruct_filename(name):
    return name.replace("_E_", "_").replace("_I_", "_") + ".wav"

In [7]:
df["filename"] = df["ID"].apply(reconstruct_filename)

In [8]:
print("📊 Dataset shape:", df.shape)
print("🔁 Label counts:\n", df["Target"].value_counts())

📊 Dataset shape: (4000, 3)
🔁 Label counts:
 Target
I    2000
E    2000
Name: count, dtype: int64


In [9]:
durations = []
rms_levels = []

for fname in df["filename"]:
    path = os.path.join(AUDIO_DIR, fname)
    y, sr = librosa.load(path, sr=SR)
    durations.append(len(y) / sr)
    rms_levels.append(np.sqrt(np.mean(y**2)))

In [10]:
df["duration"] = durations
df["rms"] = rms_levels

print("\n🕒 Duration stats (seconds):")
print(df["duration"].describe())

print("\n🔊 RMS Loudness stats:")
print(df["rms"].describe())


🕒 Duration stats (seconds):
count    4000.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: duration, dtype: float64

🔊 RMS Loudness stats:
count    4000.000000
mean        0.032345
std         0.038738
min         0.001750
25%         0.011243
50%         0.019295
75%         0.036172
max         0.434734
Name: rms, dtype: float64


=> All files: **EXACTLY 1.0 SEC LONG**

→ Skip dynamic padding/clipping logic in dataset.py...

- RMS spans 0.0017 to 0.4347, with a mean of approx 0.03

- 25% of samples are below 0.011

=> **Some recordings are very quiet**, possibly near-silent!!

-> We should: Normalize loudness, Add volume-based augmentations...